In [1]:
#EXAMPLE CODE

import socket

def get_ip_address(website_url):
    try:
        ip_address = socket.gethostbyname(website_url)
        print(f"The IP address of {website_url} is {ip_address}")
    except socket.gaierror:
        print(f"Unable to get the IP address for {website_url}")

website = input("Enter the website URL (without 'https://'): ")
get_ip_address(website)            

# gov.uk IP = 151.101.192.144
# google.com IP = 216.58.201.110
# youtube.com IP = 172.217.169.14
# gold.ac.uk = 159.100.136.66

The IP address of gold.ac.uk is 159.100.136.66


In [2]:
import ipaddress

def analyse_ip(ip_str):
    # Create an IP interface object
    ip = ipaddress.ip_interface(ip_str)

    print(f"Address: {ip.ip}")
    print(f"Network: {ip.network}")
    print(f"Netmask: {ip.netmask}")
    print(f"Number of Usable Hosts: {ip.network.num_addresses - 2}")  # Excluding network and broadcast
    print(f"Is private: {ip.ip.is_private}")
    print(f"Is global: {ip.ip.is_global}")
    print(f"Broadcast Address: {ip.network.broadcast_address}") #task 1
    print(f"First Usable Host: {list(ip.network.hosts())[0]}")
    print(f"Last Usable Host: {list(ip.network.hosts())[-1]}")    

    # List all hosts in the network (for small networks)
    if ip.network.num_addresses < 256:
        print("\nHosts in network:")
        for host in ip.network.hosts():
            print(host)

print("Analysing Campus site")
analyse_ip('159.100.136.66/24')

Analysing Campus site
Address: 159.100.136.66
Network: 159.100.136.0/24
Netmask: 255.255.255.0
Number of Usable Hosts: 254
Is private: False
Is global: True
Broadcast Address: 159.100.136.255
First Usable Host: 159.100.136.1
Last Usable Host: 159.100.136.254


| Department | Network Address | Usable IP address range| Broadcast Address | Subnet mask |
|------------|-----------------|------------------------|-------------------|-------------|
| Engineering| 172.16.0.0/27 | 172.16.0.1 to 172.16.0.30 | 172.16.0.31 | 255.255.255.224|
|Marketing	| 172.16.0.32/28| 172.16.0.33 to 172.16.0.46 | 172.16.0.47| 255.255.255.240|
|Finance	|172.16.0.48/28	|172.16.0.49 to 172.16.0.62	|172.16.0.63	|255.255.255.240
|HR	|172.16.0.64/29	|172.16.0.65 to 172.16.0.70	|172.16.0.71	|255.255.255.248


In [ ]:
# dhcp_simple.py - Simplified DHCP Simulator
#EXAMPLE CODE

#Server Configuration
server = {
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
    "leases": {}
}

# Client Configuration
client = {
    "mac": "AA:BB:CC:DD:EE:FF",
    "ip": None
}

#client sends DHCP DISCOVER
def send_discover():
    print("\n[CLIENT] Step 1: Sending DHCP DISCOVER")
    return {
        "type": "DISCOVER",
        "mac": client["mac"]
    }

#server responds with DHCP OFFER
def make_offer(discover):
    print("\n[SERVER] Step 2: Making DHCP OFFER")
    if not server["ip_pool"]:
        print("No IPs available!")
        return None
    offered_ip = server["ip_pool"].pop(0)
    return {
        "type": "OFFER",
        "mac": discover["mac"],
        "ip": offered_ip
    }

#client sends DHCP REQUEST
def send_request(offer):
    print("\n[CLIENT] Step 3: Sending DHCP REQUEST")
    return {
        "type": "REQUEST",
        "mac": offer["mac"],
        "ip": offer["ip"]
    }

#server sends DHCP ACK
def send_ack(request):
    print("\n[SERVER] Step 4: Sending DHCP ACK")
    server["leases"][request["mac"]] = request["ip"]
    return {
        "type": "ACK",
        "mac": request["mac"],
        "ip": request["ip"]
    }

# Main Function
def main():
    print("=== Simple DHCP Simulation ===")
    
    # Client starts process
    discover = send_discover()

    # Server responds
    offer = make_offer(discover)
    if not offer:
        return

    # Client continues
    request = send_request(offer)

    # Server finalizes
    ack = send_ack(request)

    # Update client IP
    client["ip"] = ack["ip"]

    # Results
    print("\n=== Result ===")
    print(f"Client {client['mac']} got IP: {client['ip']}")
    print("Server leases:", server["leases"])

# Run the program
if __name__ == "__main__":
    main()


=== Simple DHCP Simulation ===

[CLIENT] Step 1: Sending DHCP DISCOVER

[SERVER] Step 2: Making DHCP OFFER

[CLIENT] Step 3: Sending DHCP REQUEST

[SERVER] Step 4: Sending DHCP ACK

=== Result ===
Client AA:BB:CC:DD:EE:FF got IP: 192.168.1.100
Server leases: {'AA:BB:CC:DD:EE:FF': '192.168.1.100'}


In [ ]:
import socket
import json

server = {
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
    "leases": {}
}

#setting up server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(("localhost", 65432))
server_socket.listen()